#### Code By: Ryan Abramowitz

In [ ]:
import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np

folder = 'dge/' # path to folder with dge's from Dropseq alignment output
x = os.listdir(folder)
x = [folder+i for i in x]
x.sort()

""" For me, the directory has files like:
redo/star_AFE_H1_rep1_2016FEB20Dropseq_merged_gene_exon_tagged_clean.dge.txt
redo/star_AFE_H1_rep2_2016FEB18Dropseq_merged_gene_exon_tagged_clean.dge.txt
...
redo/star_PFE_H1_rep2_2016FEB23Dropseq_merged_gene_exon_tagged_clean.dge.txt
"""
for i in range(len(x)):
    print(i,x[i])
    dge = pd.read_csv(x[i],header=0,delimiter='\t',index_col='GENE')
    dge = dge.astype(pd.SparseDtype(int,fill_value=0))
    dge = dge.T
    dge.index.name = None
    dge.columns.name = None

    adata = sc.AnnData(dge)
    adata.obs['Source'] = x[i][x[i].find('star_')+5:x[i].rfind('_merged_gene_exon_tagged_clean')]
    adata.write_h5ad(f'tmp/{i}.h5ad')
    del adata

adata = ad.concat([sc.read_h5ad('tmp/'+i) for i in os.listdir('tmp')],join='outer',fill_value=0)
adata.X = csr_matrix(adata.X)
adata.obs['AFE'] = ['AFE_' in i for i in adata.obs['Source']]
adata.obs['DE'] = ['DE_' in i for i in adata.obs['Source']]
adata.obs['ESC'] = ['ESC_' in i for i in adata.obs['Source']]
adata.obs['PFE'] = ['PFE_' in i for i in adata.obs['Source']]
adata.obs['Type'] = adata.obs[['AFE','DE','ESC','PFE']].idxmax(axis=1)

In [ ]:
adata.write_h5ad('checkpoint/QCed.h5ad')

In [ ]:
import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np

adata = sc.read_h5ad('checkpoint/QCed.h5ad')
print(adata.shape)

adata.obs['Experiment'] = adata.obs['Source'].copy()
rename = {'AFE_H1_rep1_2016FEB20Dropseq':'AFE_rep1','AFE_H1_rep2_2016FEB18Dropseq':'AFE_rep2','hDE_H1_rep1_2015DEC10Dropseq':'DE_rep1','DE_H1_rep3_2016FEB18Dropseq':'DE_rep2','hESC_H1_rep1_2015DEC10Dropseq':'ESC_rep1','ESC_H1_rep2_2016FEB18Dropseq':'ESC_rep2','ESC_H1_rep3_S4_L001':'ESC_rep3','PFE_H1_rep2_2016FEB23Dropseq':'PFE_rep1',}
adata.obs['Experiment'] = adata.obs['Experiment'].astype(str).apply(lambda x: rename[x])
adata.obs['Experiment'] = adata.obs['Experiment'].astype('category').cat.reorder_categories(['ESC_rep1', 'ESC_rep2', 'ESC_rep3', 'DE_rep1', 'DE_rep2', 'AFE_rep1', 'AFE_rep2', 'PFE_rep1'], ordered=True)
adata.uns['Experiment_colors'] = ['crimson', 'darkred','gold', 'MediumAquamarine', 'CadetBlue', 'LightBlue', 'SteelBlue','#AD4E92']

adata.obs['Type'].value_counts()


In [ ]:
sc.pp.filter_cells(adata,min_counts=100)
adata.obs['doublet_score'] = 0
adata.obs['predicted_doublets'] = False
for i in set(adata.obs['Source']):
    this = adata[adata.obs['Source'] == i].copy()
    sc.external.pp.scrublet(this,batch_key='Source',verbose=False)
    adata.obs['doublet_score'].loc[adata.obs['Source'] == i] = this.obs['doublet_score']
    adata.obs['predicted_doublets'].loc[adata.obs['Source'] == i] = this.obs['predicted_doublets']
adata = adata[adata.obs['predicted_doublets'] == False]
adata.write_h5ad('checkpoint/Doublet.h5ad')
adata.obs['Type'].value_counts()


In [ ]:
adata.obs['Experiment']

In [ ]:
import scanpy as sc
import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np

adata = sc.read_h5ad('checkpoint/Doublet.h5ad')

adata.var['mt'] = adata.var.index.str.startswith('MT-')
adata.var['ribo'] = adata.var_names.str.startswith(("RPS","RPL"))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo'], percent_top=None, log1p=True, inplace=True)
import seaborn as sns

adata.obs['Good Cell'] = True
gns = [7,7,8,8,8,7.3,8,8]
cts = gns
for e,i in enumerate(sorted(list(set(adata.obs['Source'])))):
    # sns.displot(adata[adata.obs['Source'] == i].obs['log1p_total_counts'],)
    # plt.title(i)
    # plt.plot()
    # adata.obs['Good Cell'].loc[adata.obs['Source'] == i] *= adata[adata.obs['Source'] == i].obs['log1p_total_counts'] > cts[e]
    sns.displot(adata[adata.obs['Source'] == i].obs['log1p_n_genes_by_counts'],)
    plt.plot([gns[e],gns[e]],plt.gca().get_ylim(),color='r')
    plt.title(i)
    plt.show()
    # sns.displot(adata[(adata.obs['Source'] == i) & (adata.obs['pct_counts_mt'] != 0)].obs['pct_counts_mt'],)
    # plt.title(i)
    # plt.show()
    adata.obs['Good Cell'].loc[adata.obs['Source'] == i] *= adata[adata.obs['Source'] == i].obs['log1p_n_genes_by_counts'] > gns[e]
adata = adata[adata.obs['Good Cell']]
for i in ['ESC','DE','AFE','PFE']:
    # sc.pl.scatter(adata[adata.obs[i]], 'log1p_total_counts', 'log1p_total_counts_mt', color='Source')
    sc.pl.scatter(adata[adata.obs[i]], 'log1p_total_counts', 'pct_counts_mt', color='Experiment')
adata.obs['Good Cell'].loc[adata.obs['Experiment'] == 'ESC_rep1'] *= adata[adata.obs['Experiment'] == 'ESC_rep1'].obs['pct_counts_mt'] > 0.5
adata.obs['Good Cell'] *= adata.obs['pct_counts_mt'] < 10
adata = adata[adata.obs['Good Cell']]


for i in ['ESC','DE','AFE','PFE']:
    # sc.pl.scatter(adata[adata.obs[i]], 'log1p_total_counts', 'log1p_total_counts_mt', color='Source')
    sc.pl.scatter(adata[adata.obs[i]], 'log1p_total_counts', 'pct_counts_mt', color='Experiment')

adata.write_h5ad('checkpoint/ActuallyQCed.h5ad')


In [ ]:
import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np

adata = sc.read_h5ad('checkpoint/ActuallyQCed.h5ad')
adata.obs['Type'].value_counts()

In [ ]:
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata,base=np.exp(1))
adata.raw = adata

cell_cycle_genes = [x.strip() for x in open('./regev_lab_cell_cycle_genes.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
x = set(x.lower() for x in cell_cycle_genes)
cell_cycle_genes = [g for g in adata.var_names if g.lower() in x]
x = set(s.lower() for s in s_genes)
s_genes = [s for s in cell_cycle_genes if s.lower() in x]
x = set(s.lower() for s in g2m_genes)
g2m_genes = [s for s in cell_cycle_genes if s.lower() in x]
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

adata.write_h5ad('checkpoint/PPdone.h5ad')


In [ ]:
import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np
sc.set_figure_params(facecolor="white", figsize=(8, 8))
import matplotlib as mpl
plt.rcParams["figure.figsize"] = [8, 8]
plt.rcParams["figure.autolayout"] = True
sc.set_figure_params(scanpy=True, dpi_save=400,dpi=150)
mpl.rc('image', cmap = mpl.colors.LinearSegmentedColormap.from_list('super_magma', colors=['#e0e0e0', '#dedede', '#fff68f', '#ffec8b', '#ffc125', '#ee7600', '#ee5c42', '#cd3278', '#c71585', '#68228b'], N=500))
plt.rcParams["font.family"] = "Arial"
plt.rcParams['pdf.fonttype'] = 42
adata = sc.read_h5ad('checkpoint/PPdone.h5ad')

def prepare_leiden_umap(adata):
    sc.pp.highly_variable_genes(adata)
    adata.var['highly_variable'] *= np.any(adata.var[['mt', 'ribo']],axis=1) == False
    cell_cycle_genes = [x.strip() for x in open('./regev_lab_cell_cycle_genes.txt')]
    adata.var['highly_variable'] *= np.in1d(adata.var.index,cell_cycle_genes) == False
    sc.pp.pca(adata, use_highly_variable=True,n_comps=20)
    sc.external.pp.harmony_integrate(adata,'Experiment')
    sc.pp.neighbors(adata,use_rep='X_pca_harmony',)
    sc.tl.leiden(adata)

    name = adata.obs[['leiden','Type']].value_counts().unstack(level=1).idxmax(axis=1)
    adata.obs['leiden'] = adata.obs['leiden'].apply(lambda x: name.loc[x]+'-'+x).astype(str)
    namemapper = {}
    donesofar = {}
    for i in adata.obs['leiden'].value_counts().index:
        if not i[:i.find('-')] in donesofar:
            donesofar[i[:i.find('-')]] = 0
        donesofar[i[:i.find('-')]] += 1
        namemapper[i] = i[:i.find('-')]+'-'+str(donesofar[i[:i.find('-')]])
    print(namemapper)
    adata.obs['leiden'] = adata.obs['leiden'].apply(lambda x: namemapper[x]).astype(str).astype('category')
    adata.obs['leiden'] = adata.obs['leiden'].cat.reorder_categories([f'{t}-{n}' for t in adata.obs['Type'].cat.categories for n in range(len(adata.obs['leiden'].cat.categories)) if f'{t}-{n}' in adata.obs['leiden'].cat.categories], ordered=True)

    type2color = {
        'ESC':np.array((1,0,0)),
        'DE':np.array((1,1,0)),
        'AFE':np.array((0,0,1)),
        'PFE':np.array((1,0,1)),
    }
    adata.uns['leiden_colors'] = [mpl.colors.to_hex(type2color[i[:i.find('-')]]/int(i[i.find('-')+1:])) for i in adata.obs['leiden'].cat.categories]
    sc.tl.umap(adata)
    print(adata.uns['leiden_colors'])
    return None


adata.obs['Type'] = adata.obs['Type'].cat.reorder_categories(['ESC','DE','AFE','PFE'], ordered=True)
adata.obs['Type'].value_counts()

adata.uns['Type_colors'] = ['red','yellow','skyblue','rebeccapurple']
adata.uns['Experiment_colors'] = ['IndianRed', 'OrangeRed','HotPink','DarkGoldenrod', 'YellowGreen', 'DarkBlue', 'Cyan', 'darkmagenta']

prepare_leiden_umap(adata)

for i in ['Experiment','Type','leiden','log1p_n_genes_by_counts','n_genes_by_counts','phase','pct_counts_mt','doublet_score','pct_counts_ribo']:
    sc.pl.umap(adata,color=i,ncols=1,save='pre_QC'+i,add_outline=True,frameon=False)
    f = sc.pl.umap(adata,color=i,ncols=1,colorbar_loc=None,legend_loc=None,title=None,return_fig=True,add_outline=True,frameon=False)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('')
    plt.savefig('figures/umappre_QC'+i+'-no_colorbar.pdf')
    plt.savefig('figures/umappre_QC'+i+'-no_colorbar.png',dpi=600)
    plt.show()
x = adata.obs['leiden'].value_counts()
adata = adata[adata.obs['leiden'].isin(x[x > 20].index)]

prepare_leiden_umap(adata)

adata.uns['leiden_colors'] = ['crimson', 'darkred', 'coral','gold', 'MediumAquamarine', 'CadetBlue', 'LightBlue', 'SteelBlue','blue','#AD4E92']
for i in ['Experiment','Type','leiden','log1p_n_genes_by_counts','n_genes_by_counts','phase','pct_counts_mt','doublet_score','pct_counts_ribo']:
    sc.pl.umap(adata,color=i,ncols=1,save='post_QC'+i,add_outline=True,frameon=False)
    sc.pl.umap(adata,color=i,ncols=1,colorbar_loc=None,legend_loc=None,title=None,return_fig=True,add_outline=True,frameon=False)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('')
    plt.savefig('figures/umappost_QC'+i+'-no_colorbar.pdf')
    plt.savefig('figures/umappost_QC'+i+'-no_colorbar.png',dpi=600)
    plt.show()

adata.write_h5ad('checkpoint/DoneWithQC.h5ad')



In [ ]:

import os
import scanpy as sc
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import numpy as np
sc.set_figure_params(facecolor="white", figsize=(8, 8))
import matplotlib as mpl
plt.rcParams["figure.figsize"] = [8, 8]
plt.rcParams["figure.autolayout"] = True
sc.set_figure_params(scanpy=True, dpi_save=400,dpi=150)
mpl.rc('image', cmap = mpl.colors.LinearSegmentedColormap.from_list('super_magma', colors=['#e0e0e0', '#dedede', '#fff68f', '#ffec8b', '#ffc125', '#ee7600', '#ee5c42', '#cd3278', '#c71585', '#68228b'], N=500))
plt.rcParams["font.family"] = "Arial"
plt.rcParams['pdf.fonttype'] = 42


adata = sc.read_h5ad('checkpoint/DoneWithQC.h5ad')


In [ ]:
sc.pl.matrixplot(adata,adata.var_names[adata.var_names.str.startswith('HOX')],'leiden',save='All_HOX_genes_before_subsetting')
sc.pl.matrixplot(adata,adata.var_names[adata.var_names.str.startswith('HOX')][np.array(adata.X[:,adata.var_names.str.startswith('HOX')].sum(axis=0) > 5).flatten()],'leiden',standard_scale='var',colorbar_title='Scaled Expression',save='HOX_Matrix_plot_Fig_S2')


In [ ]:
this = adata[adata.obs['Type'].isin(['AFE','PFE'])]
this.raw = this.raw.to_adata()[:,this.var['ribo'] + this.var['mt'] == 0]
this = this[:,this.var['ribo'] + this.var['mt'] == 0]
sc.tl.rank_genes_groups(this,'Type',method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(this, values_to_plot='logfoldchanges', cmap='magma', vmin=-5, vmax=5, colorbar_title='log fold change',min_logfoldchange=3)


In [ ]:
sc.tl.rank_genes_groups(adata,'Type',method='wilcoxon')
sc.pl.rank_genes_groups_heatmap(adata,save='S2B-Type', show_gene_labels=True, cmap='viridis', vmin=0, dendrogram=False,min_logfoldchange=0)
sc.pl.rank_genes_groups_dotplot(adata,save='S2B-Type', values_to_plot='logfoldchanges', cmap='magma', vmin=-5, vmax=5, colorbar_title='log fold change',dendrogram=False,min_logfoldchange=0)

sc.tl.rank_genes_groups(adata,'Experiment',method='wilcoxon')
sc.pl.rank_genes_groups_heatmap(adata,save='S2B-Experiment', show_gene_labels=True, cmap='viridis', vmin=0, dendrogram=False,min_logfoldchange=0)
sc.pl.rank_genes_groups_dotplot(adata,save='S2B-Experiment', values_to_plot='logfoldchanges', cmap='magma', vmin=-5, vmax=5, colorbar_title='log fold change',dendrogram=False,min_logfoldchange=0)

sc.tl.rank_genes_groups(adata,'leiden',method='wilcoxon')
sc.pl.rank_genes_groups_heatmap(adata,save='S2B', show_gene_labels=True, cmap='viridis', vmin=0, dendrogram=False,min_logfoldchange=0)
sc.pl.rank_genes_groups_dotplot(adata,save='S2B', values_to_plot='logfoldchanges', cmap='magma', vmin=-5, vmax=5, colorbar_title='log fold change',dendrogram=False,min_logfoldchange=0)


In [ ]:


sc.pl.matrixplot(adata,['POU5F1','SOX17','VGLL2','EYA1','FOXA2','SOX2','PAX9','PAX1','FOXE1','PBX1'],'Type',save='3A_matrix',cmap=mpl.colors.LinearSegmentedColormap.from_list('super_magma', colors=['#e0e0e0', '#dedede', '#fff68f', '#ffec8b', '#ffc125', '#ee7600', '#ee5c42', '#cd3278', '#c71585', '#68228b'], N=500), standard_scale='var', colorbar_title='column scaled\nexpression')
sc.pl.matrixplot(adata,['POU5F1','SOX17','VGLL2','EYA1','FOXA2','SOX2','PAX9','PAX1','FOXE1','PBX1'],'Type',save='3A_matrix_swapped_axis',cmap=mpl.colors.LinearSegmentedColormap.from_list('super_magma', colors=['#e0e0e0', '#dedede', '#fff68f', '#ffec8b', '#ffc125', '#ee7600', '#ee5c42', '#cd3278', '#c71585', '#68228b'], N=500), standard_scale='var', colorbar_title='column scaled\nexpression',swap_axes=True)

In [ ]:

reference_markers = {
    'esc':['SOX2', 'POU5F1', 'NANOG', 'DNMT3B','ZFP42'],
    'de':['LEFTY1','LEFTY2','CER1','NODAL', 'KLF8','SOX17','GATA4','GATA6','EOMES','FGF17','CXCR4'],
    'pe':['FN1','FOXA2', 'GATA3', 'TBX1', 'GBX2', 'PAX1', 'PAX9', 'FOXA1', 'EYA1', 'NKX2-5', 'ISL1', 'SIX1', 'FGF10'],
}
sc.pl.dotplot(adata,reference_markers,'leiden',swap_axes=True,save='2C_their_genes',cmap='viridis')
sc.pl.heatmap(adata,reference_markers,'leiden',save='2C_their_genes',cmap='viridis', show_gene_labels=True)

reference_markers = {
    'esc':['NANOG', 'LEFTY1', 'POU5F1', 'SOX2'],
    'de':['CER1', 'SOX17', 'HHEX', 'LEFTY1', 'FOXA2'],
    'afe':['EYA1', 'ISL1', 'VGLL2', 'SOX2', 'FOXA2', 'RIPPLY3', 'HOXB2', 'OTX2', 'SIX1', 'TBX1', 'GATA3', 'FOXG1'],
    'pfe1':['PAX1', 'PAX9', 'DMRT2', 'EYA1', 'ISL1', 'VGLL2', 'SOX2', 'FOXA2', 'HOXA3', 'RIPPLY3', 'HOXA2', 'HOXB1', 'HOXB2', 'HOXB3', 'SIX1', 'FOXG1', 'GATA3'],
    'pfe2':['DMRT2', 'EYA1', 'ISL1', 'PAX1', 'PAX9', 'SOX2', 'SIX1', 'FOXG1', 'HOXA3', 'HOXA2', 'HOXB2', 'HOXB3', 'GATA3'],
    'uub':['CALCA', 'NKX2-1', 'ASCL1', 'MEOX1', 'RIPPLY3', 'EYA1', 'ISL1', 'PAX1', 'PAX9', 'FOXG1', 'HOXA3', 'HOXA2', 'HOXB1', 'HOXB3', 'PAX9'],
    'thymus':['PAX1', 'DMRT2', 'GAS6', 'IL7', 'PRSS16', 'CCL25', 'EYA1', 'ISL1', 'PAX9', 'SIX1', 'FOXG1', 'HOXA3', 'HOXA2', 'PAX9'],
    'parathyroid':['CASR', 'DMRT2', 'GATA3', 'VDR', 'CHGA', 'MAFB', 'PAX9', 'NGFR', 'FLRT2', 'SPARCL1', 'EYA1', 'ISL1', 'SIX1', 'PAX1', 'HOXA3', 'MEOX1', 'FOXG1', 'HOXA2', 'HOXB2', 'HOXB3', 'PAX9'],
}

sc.pl.dotplot(adata,reference_markers,'leiden',swap_axes=True,save='2C_macrina_markers',cmap='viridis')
sc.pl.heatmap(adata,reference_markers,'leiden',save='2C_macrina_markers',cmap='viridis', show_gene_labels=True)


In [ ]:
genes = adata.var_names[adata.var_names.isin(['VGLL2','SOX2','POU5F1','NANOG','DNMT3B','ZFP42','LEFTY1','LEFTY2','CER1','NODAL','KLF8','SOX17','GATA4','GATA6','EOMES','FGF17','CXCR4','FN1','FOXA2','GATA3','TBX1','GBX2','PAX1','PAX9','FOXA1','EYA1','NKX2-5','ISL1','SIX1','FGF10','HOXA-AS2','HOXA-AS3','HOXA1','HOXA10','HOXA10-AS','HOXA11','HOXA11-AS','HOXA13','HOXA2','HOXA3','HOXA4','HOXA5','HOXA6','HOXA7','HOXA9','HOXB-AS1','HOXB-AS2','HOXB-AS3','HOXB1','HOXB13','HOXB2','HOXB3','HOXB4','HOXB5','HOXB6','HOXB7','HOXB8','HOXB9','HOXC-AS2','HOXC-AS3','HOXC10','HOXC11','HOXC4','HOXC6','HOXC8','HOXC9','HOXD-AS2','HOXD1','HOXD10','HOXD13','HOXD3','HOXD4','HOXD8','HOXD9','NANOG','POU5F1','SOX2','SOX17','FOXA2','PAX1','PAX9','LEFTY1','LEFTY2','CER1','VGLL2','TBX1','HIC1','PBX1','PBX2','MEIS1','MEIS2','MEIS3','SIX1','GBX2','OTX1','OTX2','TFAP4','TFAP2C','ISL1','ISL2','IRX1','IRX2','IRX3','EYA1','FGF10','HEY1','BMP4','BMP1','BMP2', 'BMP3','BMP5','BMP6','BMP7','HES1','ID1','FN1','EOMES','NODAL','FGF17','SMAD7','FOXE1','NKX2-1','NKX2-2','NKX2-3','NKX2-5','NKX2-6','RORA','RORB','RORC','GATA3','GATA4','GATA6','TP53','TP63','SHH','DMRT2','FOXP1','FOXP2','WNT5A','ROR2'])]

for i in genes:
    sc.pl.umap(adata,color=i,ncols=1,save='S2A-'+i,show=True,add_outline=True,frameon=False)
    sc.pl.umap(adata,color=i,ncols=1,colorbar_loc=None,legend_loc=None,title=None,return_fig=True,add_outline=True,frameon=False)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('')
    plt.savefig('figures/umapS2A-'+i+'-no_colorbar.pdf')
    plt.savefig('figures/umapS2A-'+i+'-no_colorbar.png',dpi=600)
    plt.show()

genes = [i for i in genes if adata[:,i].X.sum() > 0]

adata_magic = sc.external.pp.magic(adata,genes,t=3,random_state=0,copy=True)
adata_magic.obsm['X_umap'] = adata.obsm['X_umap']
del adata_magic.raw

for i in genes:
    sc.pl.umap(adata_magic,color=i,ncols=1,save='S2A-'+i+'-MAGIC-Impute',show=True,add_outline=True,frameon=False)
    sc.pl.umap(adata_magic,color=i,ncols=1,colorbar_loc=None,legend_loc=None,title=None,return_fig=True,add_outline=True,frameon=False)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('')
    plt.savefig('figures/umapS2A-'+i+'-no_colorbar-MAGIC-Impute.pdf')
    plt.savefig('figures/umapS2A-'+i+'-no_colorbar-MAGIC-Impute.png',dpi=600)
    plt.show()


In [ ]:
adata.write_h5ad('checkpoint/END_OF_ANALYSIS.h5ad')